In [ ]:
%pip install -i https://test.pypi.org/simple/ --extra-index-url=https://pypi.org/simple azure-datalake-utils==0.2.0.dev23

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
%pip uninstall azure-datalake-utils -y

Found existing installation: azure-datalake-utils 0.5.1
Uninstalling azure-datalake-utils-0.5.1:
  Successfully uninstalled azure-datalake-utils-0.5.1
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip show azure-datalake-utils

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys

import pandas as pd

sys.path.append('../')

In [3]:
from azure_datalake_utils import Datalake

In [4]:
import json

with open('../keys.json') as f:
    creds = json.load(f)


## reproducir el caso `BadZipFile`

Se observa un comportamiento extraño bajo estas condiciones:

1. Se importa la libreria en un sesion de python.
1. Se abre el archivo de excel y se reemplaza en el datalake.
1. Usando la misma sesión de python, se trata de leer el archivo con `dl.read_excel("hacebanalitica-user-servicio/configuracion_valores_salesforce.xlsx")`
1. Una exepción `BadZipFile: File is not a zip file`.

WorArounds:
1. Reiniciar la sesión de python. Solo realizar esa acción ya el error no aparece.
2. usar el `flag=True` ejemplo:

```python
excel = dl.read_excel("hacebanalitica-user-servicio/configuracion_valores_salesforce.xlsx", sheet_name = 'familia', experimental = True)
```
Este flag es experimental y es una solución rapida para entornos donde reiniciar python de manera periodica no es una opción (ejemplo an Azure Functions.)

In [5]:
dl = Datalake.from_account_key(**creds)

In [ ]:
# modificar el archivo de excel a leer y subirlo a la misma ruta.
excel = dl.read_excel("hacebanalitica-user-servicio/configuracion_valores_salesforce.xlsx", sheet_name = 'familia')

In [15]:
# si es usa el flag, ya no hay falla.
excel = dl.read_excel("hacebanalitica-user-servicio/configuracion_valores_salesforce.xlsx", sheet_name = 'familia', experimental = True)

## Uso normal.

In [5]:
dl = Datalake.from_account_key(**creds, fsspec_cache = False)

In [6]:
df = pd.DataFrame({"foo_str": ['bar\n', 'foo,', 'bar|'], "bar_str": ["bar", "foo|", "bar\r"], "foo": [1, 2, 3]})

In [7]:
dl.write_csv(df, 'hacebanalitica-user-cientificos/prueba/test.csv', sep='|')

In [32]:
test = dl.read_csv('hacebanalitica-user-cientificos/prueba/test.csv', header = None,  sep='|')

In [ ]:
test = dl.read_csv('hacebanalitica-curated-ventas/ventas/haceb/ventas/ventas_2022.csv', sep=';')

In [12]:
test = dl.read_csv('hacebanalitica/raw/sap/ztsd_clientepos/load_date=2022-05-04/ztsd_clientepos.csv', sep='|')

/home/german/repos/azure-datalake-utils/docs/../azure_datalake_utils/azure_datalake_utils.py:80: DtypeWarning: Columns (10,16,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"az://{ruta}", storage_options=self.storage_options, **kwargs)


In [14]:
test.head()

,MANDT,FITYP,STCDT,STCD1,KUNNR,ANRED,NAME1,STREET,CITY1,COUNTRY,...,ERNAM,AEDAT,AEZET,UNAM,IN_CLISAP,CSTCNDGRP1,ZSD_HABEASDATA,TRANSPZONE,LANDX,KTOKD
0,300,PN,13,632453,632453,Señor(a),GUSTAVO PARRA,CR 45 A 80 SUR 75,MEDELLIN,CO,...,LUISZAP,2018.03.28,07:31:51,LUISZAP,2,,1,5001,,
1,300,PN,13,63311387,63311387,Señor(a),ALMANZAR SANCHEZ,CORDOBA,MEDELLIN,CO,...,LUISZAP,2018.03.28,07:31:51,LUISZAP,2,,1,5001,,
2,300,PN,13,6819796,6819796,Señor(a),FERNANDO BOTERO,CR 81 B 7 A 49,MEDELLIN,CO,...,LUISZAP,2018.03.28,07:31:51,LUISZAP,2,,1,5001,,
3,300,PN,13,70030346,70030346,Señor(a),ALBERTO LONDONO,CLIENTE TRNASPORTA,MEDELLIN,CO,...,LUISZAP,2018.03.28,07:31:51,LUISZAP,2,,1,5001,,
4,300,PN,13,70040014,70040014,Señor(a),PEDRO HERNANDEZ,COTIZACION,MEDELLIN,CO,...,LUISZAP,2018.03.28,07:31:51,LUISZAP,2,,1,5001,,


### Interactivo

In [15]:
dl = Datalake('datalakeanaliticabi', '5368fc5a-2692-4a6a-b226-e2802e94e22b')

In [16]:
test = dl.read_csv('hacebanalitica-user-cientificos/prueba/*.csv', header = None)

In [ ]:
test = dl.read_csv('hacebanalitica/raw/sap/ztsd_clientepos/load_date=2022-05-04/ztsd_clientepos.csv', sep='|')

## Como usar las particiones

La libreria tiene la funcionalidad de leer las particiones. Tipo Hive, solo se soporta esta funcionalidad cuando es incializado por account key.

In [11]:
dl = Datalake.from_account_key(**creds)

In [16]:
# la libreria descubre las particiones.
# filtrando la particion final , para obtener el ultimo valor.
users = dl.read_csv_with_partition("hacebanalitica/raw/tuya_cloud/users/", last_modified_last_level=True, sep = "|")

In [19]:
display("concatenacion de todos", users.sample(5), users.shape, "se incluye la partición inferida", users.start_date.unique())

'concatenacion de todos'

,avatar,country_code,create_time,email,mobile,nick_name,temp_unit,time_zone_id,uid,update_time,user_properties,username,start_date
21089,NaN,57,2022-10-13 04:06:57,osnaiderwendy18@gmail.com,NaN,NaN,1,America/Bogota,az16656340174644JPxY,2022-10-13 04:07:18,"[{'code': 'dataAuthorization', 'value': '1', '...",245861383be64718bab52394555c1888,2022_11_27
21712,NaN,57,2022-05-15 02:23:19,salgadosare@gmail.com,NaN,NaN,1,America/Bogota,az1652581399656TOcsE,2022-05-15 02:23:30,"[{'code': 'dataAuthorization', 'value': '1', '...",f9b2c9abb2e7429c83cd1284eba950a2,2022_11_26
4891,NaN,57,2022-11-14 23:15:49,rodrigo.17arroyo@hotmail.com,NaN,NaN,1,America/Bogota,az1668467749587ZgFiE,2022-11-14 23:16:12,"[{'code': 'dataAuthorization', 'value': '1', '...",75a57b761a524a18bc9aca904137ef27,2022_12_07
37791,NaN,57,2022-09-03 19:42:30,primeroerg@gmail.com,NaN,NaN,1,America/Bogota,az1662234150190Lj3rg,2022-09-03 19:42:42,"[{'code': 'dataAuthorization', 'value': '1', '...",a5bdf07b25404f9cae077e907eeb8b91,2022_11_16
28045,NaN,57,2022-06-06 03:02:20,jaimeraulfuentes@gmail.com,NaN,NaN,1,America/Bogota,az16544845402385rW9T,2022-06-06 03:02:28,"[{'code': 'dataAuthorization', 'value': '1', '...",eba1e0bd1e5f4cb9a94a6e4e8afa1dc9,2022_11_22


(72230, 13)

'se incluye la partición inferida'

array(['2022_12_09', '2022_12_08', '2022_12_07', '2022_12_06',
       '2022_12_05', '2022_12_04', '2022_12_03', '2022_12_02',
       '2022_12_01', '2022_11_30', '2022_11_29', '2022_11_28',
       '2022_11_27', '2022_11_26', '2022_11_25', '2022_11_24',
       '2022_11_23', '2022_11_22', '2022_11_21', '2022_11_20',
       '2022_11_19', '2022_11_18', '2022_11_17', '2022_11_16',
       '2022_11_15', '2022_11_14', '2022_11_13', '2022_11_12',
       '2022_11_11', '2022_11_10', '2022_11_09', '2022_11_08',
       '2022_11_07', '2022_11_06', '2022_11_05', '2022_11_04',
       '2022_11_03', '2022_11_02', '2022_11_01', '2022_10_31',
       '2022_10_30', '2022_10_29', '2022_10_28', '2022_10_27',
       '2022_10_26', '2022_10_25', '2022_10_24', '2022_09_24'],
      dtype=object)

In [20]:
# la libreria descubre las particiones, de toda la estructura
users = dl.read_csv_with_partition("hacebanalitica/raw/tuya_cloud/users/", last_modified_last_level=False, sep = "|")

In [21]:
display("concatenacion de todos", users.sample(2), users.shape, "se incluye la partición inferida", users.start_date.unique(), users.load_date.unique())

'concatenacion de todos'

,avatar,country_code,create_time,email,mobile,nick_name,temp_unit,time_zone_id,uid,update_time,user_properties,username,start_date,load_date
63973,NaN,57,2022-09-29 01:52:29,juancarlosmorales504@gmail.com,NaN,NaN,1,America/Bogota,az1664416349049G4yre,2022-09-29 01:52:41,"[{'code': 'dataAuthorization', 'value': '1', '...",4e98bb7c22e44cfda95cd287bc373b31,2022_10_29,2022_10_30
60410,NaN,57,2022-05-17 03:06:31,andres.vargas@haceb.com,NaN,ANDRESVAR,1,America/Bogota,az1652756791761O1izO,2022-05-17 04:32:24,"[{'code': 'dataAuthorization', 'value': '1', '...",1036926691,2022_10_31,2022_11_01


(72230, 14)

'se incluye la partición inferida'

array(['2022_12_09', '2022_12_08', '2022_12_07', '2022_12_06',
       '2022_12_05', '2022_12_04', '2022_12_03', '2022_12_02',
       '2022_12_01', '2022_11_30', '2022_11_29', '2022_11_28',
       '2022_11_27', '2022_11_26', '2022_11_25', '2022_11_24',
       '2022_11_23', '2022_11_22', '2022_11_21', '2022_11_20',
       '2022_11_19', '2022_11_18', '2022_11_17', '2022_11_16',
       '2022_11_15', '2022_11_14', '2022_11_13', '2022_11_12',
       '2022_11_11', '2022_11_10', '2022_11_09', '2022_11_08',
       '2022_11_07', '2022_11_06', '2022_11_05', '2022_11_04',
       '2022_11_03', '2022_11_02', '2022_11_01', '2022_10_31',
       '2022_10_30', '2022_10_29', '2022_10_28', '2022_10_27',
       '2022_10_26', '2022_10_25', '2022_10_24', '2022_09_24'],
      dtype=object)

array(['2022_12_10', '2022_12_09', '2022_12_08', '2022_12_07',
       '2022_12_06', '2022_12_05', '2022_12_04', '2022_12_03',
       '2022_12_02', '2022_12_01', '2022_11_30', '2022_11_29',
       '2022_11_28', '2022_11_27', '2022_11_26', '2022_11_25',
       '2022_11_24', '2022_11_23', '2022_11_22', '2022_11_21',
       '2022_11_20', '2022_11_19', '2022_11_18', '2022_11_17',
       '2022_11_16', '2022_11_15', '2022_11_14', '2022_11_13',
       '2022_11_12', '2022_11_11', '2022_11_10', '2022_11_09',
       '2022_11_08', '2022_11_07', '2022_11_06', '2022_11_05',
       '2022_11_04', '2022_11_03', '2022_11_02', '2022_11_01',
       '2022_10_31', '2022_10_30', '2022_10_29', '2022_10_28',
       '2022_10_27', '2022_10_26', '2022_10_25', '2022_10_24'],
      dtype=object)

In [22]:
# la libreria descubre las particione y excluye las que no necesita.
fechas_a_filtrar = ['2022_12_09', '2022_12_08', '2022_12_07', '2022_12_06',
       '2022_12_05', '2022_12_04', '2022_12_03', '2022_12_02']
users = dl.read_csv_with_partition("hacebanalitica/raw/tuya_cloud/users/", last_modified_last_level=True, sep = "|", partition_exclusion={'start_date': fechas_a_filtrar})

In [24]:
'2022_12_03' in users.start_date.unique()

False

In [25]:
'2022_12_02' in users.start_date.unique()

False

In [26]:
# O incluye las que se desea
fechas_a_filtrar = ['2022_12_09', '2022_12_08', '2022_12_07']
users = dl.read_csv_with_partition("hacebanalitica/raw/tuya_cloud/users/", last_modified_last_level=True, sep = "|", partition_inclusion={'start_date': fechas_a_filtrar})

In [29]:
users.start_date.unique()

array(['2022_12_09', '2022_12_08', '2022_12_07'], dtype=object)

In [30]:
# Si se sabe a priori la estrctura de las particiones
# se pueden especificar los valores que se quiere.
# esta es la opción mas rapida, ya que se evita el auto descubrimiento
# y se lee la particiones que se quieren.
cols = {'start_date': ['2022_12_09', '2022_12_08']}
users = dl.read_csv_with_partition("hacebanalitica/raw/tuya_cloud/users/", last_modified_last_level=True, sep = "|", partition_cols=cols)

In [32]:
users.start_date.unique()

array(['2022_12_09', '2022_12_08'], dtype=object)

## Probar el uso del FS

In [4]:
from adlfs import AzureBlobFileSystem

In [5]:
fs = AzureBlobFileSystem(account_name=creds['datalake_name'], account_key=creds['account_key'])

In [ ]:
fs.find("hacebanalitica/raw/tuya_cloud/devices/")[0:10]